# Multi-agent Collaboration for Financial Analysis

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [ ]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [ ]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "This task involves analyzing historical and real-time market data for AAPL using statistical modeling and machine learning techniques to identify trends and predict market movements."}

> Entering new CrewAgentExecutor chain...
I need to gather both historical and real-time market data for AAPL to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://fi

Final Answer: 
The current stock price for Apple Inc. (AAPL) is $231.32 with a 0.36% increase. The market cap is $3.433 trillion, and the company offers a range of products including smartphones, personal computers, tablets, wearables, and accessories. AAPL has a P/E ratio of 35.10, EPS of 6.59, and a forward dividend yield of 0.45%. The company's recent performance shows a YTD return of 7.30% and a 1-year return of 2.61%. The stock is traded on the NasdaqGS exchange.

> Finished chain.
 

The current stock price for Apple Inc. (AAPL) is $231.32 with a 0.36% increase. The market cap is $3.433 trillion, and the company offers a range of products including smartphones, personal computers, tablets, wearables, and accessories. AAPL has a P/E ratio of 35.10, EPS of 6.59, and a forward dividend yield of 0.45%. The company's recent performance shows a YTD return of 7.30% and a 1-year return of 2.61%. The stock is traded on the NasdaqGS exchange.

Thought: 
I need to gather insights and alerts

Final Answer: Based on the analysis of historical and real-time market data for AAPL, the stock price for Apple Inc. (AAPL) is currently at $231.32, with a slight increase of 0.36%. The stock has shown a 1-year return of 2.61% and a 5-year return of 90.54%. With a market cap of $3.433T, AAPL remains a significant player in the consumer electronics industry. The company offers a range of products, including smartphones, personal computers, tablets, wearables, and accessories, and operates various platforms such as the App Store and Apple Music. The profitability and financial highlights of AAPL indicate a strong performance, with a profit margin of 24.30% and a return on equity of 149.81%. Overall, the analysis suggests that AAPL continues to be a stable investment option with potential growth opportunities in the consumer electronics sector.

> Finished chain.
 

Based on the analysis of historical and real-time market data for AAPL, the stock price for Apple Inc. (AAPL) is currently a

Final Answer: 
After conducting a search on potential trading strategies for AAPL based on historical performance, I found several insightful resources. These include swing trading strategies, moving averages analysis, effective trading strategies, and options trading strategies specifically tailored for AAPL. Additionally, there are references to AAPL's historical stock performance and market responses to events such as the Death Cross.

Considering the user-defined risk tolerance of medium and trading preference for day trading, it would be beneficial to focus on short-term trading strategies that capitalize on AAPL's price movements within a single trading day. Utilizing technical analysis indicators such as moving averages, swing trading strategies, and options trading strategies can help optimize trading decisions based on historical performance trends.

Overall, a combination of technical analysis tools and short-term trading strategies aligned with historical stock performance i